In [11]:
from openai import OpenAI
import json
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

In [12]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

In [13]:
import requests
import json

def make_http_request(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text  
        else:
            return f"Request failed with status code: {response.status_code}"
    except requests.RequestException as e:
        return f"An error occurred: {e}"
    
def get_endpoints(endpoint_path='./endpoints.json'):
    with open(endpoint_path, 'r') as f:
        return json.load(endpoint_path)

In [14]:
assistant = client.beta.assistants.create(
    name='GIS Assistant',
    instructions='You are a GIS assistant, and will perform and answer questions about geospatial analysis.',
    model='gpt-4-1106-preview',
)

assistant = client.beta.threads.create_and_run(assistant_id=assistant.id)

show_json(assistant)

{'id': 'run_MoIDbJOOCOdIU7m0eIMsBAyS',
 'assistant_id': 'asst_WtTIJKmetUO3R3gNuddeKbuO',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1702465294,
 'expires_at': 1702465894,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a GIS assistant, and will perform and answer questions about geospatial analysis.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_kI92HVvfaVwMPp1czb5YeATv',
 'tools': []}

In [16]:
endpoint_file = client.files.create(
    file=open(
        './endpoints.json',
        'rb'
    ),
    purpose="assistants"
)

assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    file_ids=[endpoint_file.id],
)

show_json(assistant)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid file format. Supported formats: ['c', 'cpp', 'csv', 'docx', 'html', 'java', 'json', 'md', 'pdf', 'php', 'pptx', 'py', 'rb', 'tex', 'txt', 'css', 'jpeg', 'jpg', 'js', 'gif', 'png', 'tar', 'ts', 'xlsx', 'xml', 'zip']", 'type': 'invalid_request_error', 'param': None, 'code': None}}